In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit, cross_validate
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [2]:
selected_columns = [
    "CMRR", "PRSUP", "depth_of_ cover",
    "intersection_diagonal", "mining_hight",
    "roof_fall_rate", "fall"
]

df = pd.read_csv("original_data.csv")
df = df[selected_columns]
df.head()


,CMRR,PRSUP,depth_of_ cover,intersection_diagonal,mining_hight,roof_fall_rate,fall
0,59,5.32,150,65.9,6.0,0.00,0
1,59,5.32,150,65.9,6.0,0.00,0
2,50,3.93,400,60.0,7.0,0.66,1
3,50,5.90,400,60.0,7.0,1.08,1
4,75,3.93,400,60.0,7.0,0.00,0


In [3]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in split.split(df, df["fall"]):
    train = df.loc[train_idx]
    test  = df.loc[test_idx]

X_train = train.drop(["roof_fall_rate", "fall"], axis=1)
y_train = train["roof_fall_rate"]

X_test  = test.drop(["roof_fall_rate", "fall"], axis=1)
y_test  = test["roof_fall_rate"]


In [4]:
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler())
])

X_train_prep = num_pipe.fit_transform(X_train)
X_test_prep  = num_pipe.transform(X_test)


## lightgbm model

In [6]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------------------------------ --- 1.3/1.5 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.1 MB/s eta 0:00:00


In [5]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(
    n_estimators=500,   # number of boosting trees → more trees = better learning but slower
    learning_rate=0.05, # how fast model learns → smaller = slower but safer
    max_depth=-1,       # -1 = no depth limit → tree can grow as needed
    random_state=42     # seed to ensure reproducible results
)

lgbm.fit(X_train_prep, y_train)


ModuleNotFoundError: No module named 'lightgbm'